In [1]:
from benchmark import impl
from benchmark import alphaxutils
from decimal import Decimal
import pandas as pd

Order = alphaxutils.Order
tradeBySourceAmount = alphaxutils.tradeBySourceAmount
tradeByTargetAmount = alphaxutils.tradeByTargetAmount
AlphaRouter = alphaxutils.AlphaRouter
CarbonOrderUI = alphaxutils.CarbonOrderUI
assertAlmostEqual = alphaxutils.assertAlmostEqual
get_geoprice = AlphaRouter.get_geoprice

In [2]:
threshold_orders = 6
inputAmount = Decimal('10000') # by Target = USDC amount
support_partial = True

order_params = {
    'liquidity':Decimal('5'),
    'highestRate':Decimal('0.0005'), # 2000
    'lowestRate':Decimal('0.0004'),  # 2500
    'marginalRate':Decimal('0.0005'),
    }
order_params2 = {
    'liquidity':Decimal('4'),
    'highestRate':Decimal('0.0006'), # 1666
    'lowestRate':Decimal('0.0005'),  # 2000
    'marginalRate':Decimal('0.0006'),
    }

In [3]:
orders = []
for i in range(5):
    orders += [Order(order_params)]
    orders += [Order(order_params2)]

indexes = list(range(len(orders)))   

orders[0].z

Decimal('5')

In [4]:
total_liquidity = sum(o.y for o in orders)
total_liquidity

Decimal('45')

# Trade Functions
- Note they use pure math so will exceed the liquidity of an order

In [5]:
# trade 1 ETH into the first order 
tradeBySourceAmount(1, orders[0])

(1, Decimal('0.00049999472141527749746388041246312117636319382087660'))

In [6]:
# trade out exactly 2374.6507262194609015632130899344576184001508194173... USDC
tradeByTargetAmount(Decimal('2374.6507262194609015632130899344576184001508194173'), orders[0])

(Decimal('-96648.952539255305385209096450200859865898827170900'),
 Decimal('2374.6507262194609015632130899344576184001508194173'))

# Match by Target

In [7]:
def matchByTarget_selector(inputAmount, orders):
    hypothetical_output_amts = {i: tradeBySourceAmount(x=inputAmount, order=orders[i])[1] for i in indexes}   # WTF not sure why this is bySource but it works - for input USDC as target, orders defined in ETH per USDC
    # max_output_amt = {i: tradeByTargetAmount(x=orders[i].y, order=orders[i])[1] for i in indexes}  
    associated_liquidity = [orders[i].y for i in hypothetical_output_amts.keys()]

    amounts = []
    effective_prices = []
    available_value = []
    for k, v in hypothetical_output_amts.items():
        if v > orders[k].y:
            # print('1')
            price = get_geoprice(k, orders)
            amount = orders[k].y
            amounts += [amount]
            effective_prices += [price]
            available_value += [amount / price]
        else:
            # print('2')
            amounts += [v]
            price = v / inputAmount
            effective_prices += [price]
            available_value += [v / price]

    results = pd.DataFrame(
        [
            hypothetical_output_amts.keys(),
            hypothetical_output_amts.values(),
            associated_liquidity,
            amounts,
            effective_prices,
            available_value,
        ],
        index=[
            "indexes",
            "hypothetical_output_amts",
            "associated_liquidity",
            "amount",
            "effective_prices",
            "available_value",
        ],
    )
    results = results.T.copy()
    results.sort_values(
        by=["hypothetical_output_amts", 'indexes'], ascending=[False, True], inplace=True
    )

    results.fillna(0, inplace=True)
    results.reset_index(inplace=True, drop=True)
    # results

    if (not support_partial) & (results.available_value.sum() < abs(inputAmount)):
        print('Insufficient Liquidity')
        return(None)
    else:
        passed_indexes = AlphaRouter.gen_one_order_selector(results.available_value, abs(inputAmount), threshold_orders)
        top_n_threshold_orders = [results.indexes[i] for i in passed_indexes]
    order_subset = [orders[i] for i in top_n_threshold_orders]
    total_subset_liquidity = results[results.indexes.isin(top_n_threshold_orders)].available_value.sum()
    return(order_subset, total_subset_liquidity, top_n_threshold_orders)

In [8]:
order_subset, total_subset_liquidity, top_n_threshold_orders = matchByTarget_selector(inputAmount, orders)

In [9]:
def mpr_matchByTarget(inputAmount, order_subset, total_subset_liquidity, top_n_threshold_orders, support_partial):
    print('inputAmount', inputAmount)
    print('total_subset_liquidity', total_subset_liquidity)
    if inputAmount == total_subset_liquidity:
            rl1 = [o.y for o in order_subset]
            rl2 = [o.dxfromdy_f(o.y) for o in order_subset]
    elif inputAmount > total_subset_liquidity:
        if support_partial:
            print(f'** Partial Match ({total_subset_liquidity/inputAmount*100:0.5f}%) **')
            inputAmount = total_subset_liquidity
            rl1 = [o.y for o in order_subset]
            rl2 = [o.dxfromdy_f(o.y) for o in order_subset]
        else:
            print('Insufficient Liquidity with threshold orders')
            return(None)
    else:
        dy_f = lambda p: sum(o.dyfromp_f(p) for o in order_subset)
        dx_f = lambda p: sum(o.dxfromdy_f(o.dyfromp_f(p)) for o in order_subset)
        p_goal = CarbonOrderUI.goalseek(lambda p: dx_f(p)-inputAmount, Decimal('0.000000001'), Decimal('1000000000'))
        rl1 = [o.dyfromp_f(p_goal) for o in order_subset]
        rl2 = [o.dxfromdy_f(o.dyfromp_f(p_goal)) for o in order_subset]

    actions = {top_n_threshold_orders[i]:{"input":rl2[i],"output":rl1[i]} for i in range(len(top_n_threshold_orders))}
    assertAlmostEqual(inputAmount, sum(rl2), Decimal('1E-8'))
    print('total_input',sum(rl2))
    print('total_output', sum(rl1))
    print('effective_price', sum(rl1) / sum(rl2))
    print('1/effective_price', sum(rl2) / sum(rl1))
    return(actions)

mpr_matchByTarget(inputAmount, order_subset, total_subset_liquidity, top_n_threshold_orders, support_partial)

inputAmount 10000
total_subset_liquidity 46514.837167011074230464652186720142263516312999865
total_input 10000.000001030403082352972439714672115593227025472
total_output 5.8471626461075785995833589625529708874172614497215
effective_price 0.00058471626455050851582885989167364252311600903374859
1/effective_price 1710.2312020835171419428757728486166928460220662765


{1: {'input': Decimal('2000.0000002060806164705944879429344231186454050944'),
  'output': Decimal('1.1694325292215157199166717925105941774834522899443')},
 3: {'input': Decimal('2000.0000002060806164705944879429344231186454050944'),
  'output': Decimal('1.1694325292215157199166717925105941774834522899443')},
 5: {'input': Decimal('2000.0000002060806164705944879429344231186454050944'),
  'output': Decimal('1.1694325292215157199166717925105941774834522899443')},
 7: {'input': Decimal('2000.0000002060806164705944879429344231186454050944'),
  'output': Decimal('1.1694325292215157199166717925105941774834522899443')},
 9: {'input': Decimal('2000.0000002060806164705944879429344231186454050944'),
  'output': Decimal('1.1694325292215157199166717925105941774834522899443')},
 0: {'input': Decimal('0E-46'), 'output': 0}}

# Match by Source

In [10]:
def matchBySource_selector(inputAmount, orders):
    hypothetical_output_amts = {i: tradeByTargetAmount(x=inputAmount, order=orders[i])[0] for i in indexes}   #WTF not sure why this is bySource
    max_output_amt = {i: tradeByTargetAmount(x=orders[i].y, order=orders[i])[0] for i in indexes}  
    ordered_associated_liquidity = {i:orders[i].y for i in hypothetical_output_amts.keys()}

    results = pd.DataFrame(
        [
            hypothetical_output_amts.keys(),
            hypothetical_output_amts.values(),
            max_output_amt.keys(),
            max_output_amt.values(),
        ],
        index=[
            "indexes",
            "hypothetical_output_amts",
            "indexes_b",
            "max_output_amt",
        ],
    )
    results = results.T.copy()
    assert(list(results.indexes) == list(results.indexes_b))

    results.sort_values(
        by=["hypothetical_output_amts", 'indexes'], ascending=[True, True], inplace=True
    )

    results2 = pd.DataFrame(
        [
            ordered_associated_liquidity.keys(),
            ordered_associated_liquidity.values(),
        ],
        index=[
            "ordered_associated_liquidity_keys",
            "ordered_associated_liquidity",
        ],
    )
    results2 = results2.T.copy()

    results = pd.merge(results, results2, how='left', left_on = 'indexes', right_on='ordered_associated_liquidity_keys')
    assert(list(results.indexes) == list(results.ordered_associated_liquidity_keys))

    results.fillna(0, inplace=True)
    results.reset_index(inplace=True, drop=True)
    # results

    if (not support_partial) & (results.ordered_associated_liquidity.sum() < abs(inputAmount)):
        print('Insufficient Liquidity')
        return(None)
    else:
        passed_indexes = AlphaRouter.gen_one_order_selector(results.ordered_associated_liquidity, abs(inputAmount), threshold_orders)
        top_n_threshold_orders = [results.indexes[i] for i in passed_indexes]
    order_subset = [orders[i] for i in top_n_threshold_orders]
    total_subset_liquidity = results[results.indexes.isin(top_n_threshold_orders)].ordered_associated_liquidity.sum()
    return(order_subset, total_subset_liquidity, top_n_threshold_orders)

In [11]:
order_subset, total_subset_liquidity, top_n_threshold_orders = matchBySource_selector(inputAmount, orders)

In [12]:
def mpr_matchBySource(inputAmount, order_subset, total_subset_liquidity, top_n_threshold_orders, support_partial):    
    print('inputAmount', inputAmount)
    print('total_subset_liquidity', total_subset_liquidity)
    if inputAmount == total_subset_liquidity:
            rl1 = [o.y for o in order_subset]
            rl2 = [o.dxfromdy_f(o.y) for o in order_subset]
    elif inputAmount > total_subset_liquidity:
        if support_partial:
            print(f'** Partial Match ({total_subset_liquidity/inputAmount*100:0.5f}%) **')
            inputAmount = total_subset_liquidity
            rl1 = [o.y for o in order_subset]
            rl2 = [o.dxfromdy_f(o.y) for o in order_subset]
        else:
            print('Insufficient Liquidity with threshold orders')
            return(None)
    else:
        dy_f = lambda p: sum(o.dyfromp_f(p) for o in order_subset)
        dx_f = lambda p: sum(o.dxfromdy_f(o.dyfromp_f(p)) for o in order_subset)
        p_goal = CarbonOrderUI.goalseek(lambda p: dy_f(p)-inputAmount, Decimal('0.000000001'), Decimal('1000000000'))
        rl1 = [o.dyfromp_f(p_goal) for o in order_subset]
        rl2 = [o.dxfromdy_f(o.dyfromp_f(p_goal)) for o in order_subset]

    actions = {top_n_threshold_orders[i]:{"input":rl1[i],"output":rl2[i]} for i in range(len(top_n_threshold_orders))}
    assertAlmostEqual(inputAmount, sum(rl1), Decimal('1E-8'))
    print('total_input',sum(rl1))
    print('total_output', sum(rl2))
    print('effective_price', sum(rl1) / sum(rl2))
    print('1/effective_price', sum(rl2) / sum(rl1))
    return(actions)
    
mpr_matchBySource(inputAmount, order_subset, total_subset_liquidity, top_n_threshold_orders, support_partial)

inputAmount 10000
total_subset_liquidity 29
** Partial Match (0.29000%) **
total_input 29
total_output 63204.666870896957256322272155625934338718721590268
effective_price 0.00045882687838915354292782070335774230457929040542126
1/effective_price 2179.4712714102399053904231777802046323696110893196


{0: {'input': Decimal('5'),
  'output': Decimal('11180.339887498948482045868343656381177203091798059')},
 2: {'input': Decimal('5'),
  'output': Decimal('11180.339887498948482045868343656381177203091798059')},
 4: {'input': Decimal('5'),
  'output': Decimal('11180.339887498948482045868343656381177203091798059')},
 6: {'input': Decimal('5'),
  'output': Decimal('11180.339887498948482045868343656381177203091798059')},
 8: {'input': Decimal('5'),
  'output': Decimal('11180.339887498948482045868343656381177203091798059')},
 1: {'input': Decimal('4'),
  'output': Decimal('7302.9674334022148460929304373440284527032625999732')}}